# Tourism Package Purchase Prediction - MLOps Project

## Business Context

"Visit with us" is a travel and tourism company that offers a variety of travel packages for destinations worldwide. The company's latest offering is the "Wellness Tourism Package," designed to promote health and well-being through travel experiences.

The marketing team has been running campaigns to promote this new package, but the conversion rate has been lower than expected. To improve targeting and increase conversions, the company wants to predict which customers are most likely to purchase the Wellness Tourism Package based on their demographics, travel preferences, and past interactions with the company.

## Objective

Build an automated MLOps pipeline using GitHub Actions and Hugging Face to:
- Register and prepare tourism customer data
- Train multiple ML models with MLflow experiment tracking
- Deploy the best model as a Streamlit web application
- Automate the entire pipeline with CI/CD

## Prerequisites

1. **Create a GitHub Repository**
   - Go to GitHub Profile → Your repositories → New
   - Repository Name: `MLOps`
   - Check the box for `README.md` file
   - Click Create repository

2. **Add Hugging Face Token to GitHub Secrets**
   - Go to Hugging Face Profile → Access Token
   - Create New token (Write access)
   - Copy the generated token
   - In GitHub repo: Settings → Secrets and Variables → Actions
   - Add Repository secret: Name = `HF_TOKEN`, Secret = (paste token)

3. **Create Hugging Face Space**
   - Go to Hugging Face → Profile → New Space
   - Space name: `tourism-package-prediction-app`
   - Select SDK: Docker
   - Choose template: Streamlit
   - Click Create Space

# Create Master Folder Structure

In [ ]:
import os

os.makedirs("tourism_project", exist_ok=True)
print("Created tourism_project folder")

# Data Registration

In [ ]:
os.makedirs("tourism_project/data", exist_ok=True)
print("Created tourism_project/data folder")
print("Please upload tourism.csv to the tourism_project/data folder before proceeding")

In [ ]:
%%writefile tourism_project/data_registration.py
import os
from huggingface_hub import HfApi, login
import pandas as pd

def register_dataset():
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
        print("Successfully logged in to Hugging Face")
    else:
        print("Warning: HF_TOKEN not found")
        return
    
    api = HfApi()
    repo_id = "tourism-package-prediction-data"
    repo_type = "dataset"
    
    try:
        api.create_repo(repo_id=repo_id, repo_type=repo_type, exist_ok=True, private=False)
        print(f"Repository '{repo_id}' created/verified")
        
        dataset_path = "tourism_project/data/tourism.csv"
        if os.path.exists(dataset_path):
            api.upload_file(
                path_or_fileobj=dataset_path,
                path_in_repo="tourism.csv",
                repo_id=repo_id,
                repo_type=repo_type
            )
            print(f"Dataset uploaded successfully to {repo_id}")
            df = pd.read_csv(dataset_path)
            print(f"Dataset shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
        else:
            print(f"Error: Dataset file not found at {dataset_path}")
    except Exception as e:
        print(f"Error during data registration: {str(e)}")
        raise

if __name__ == "__main__":
    register_dataset()

# Data Preparation

In [ ]:
%%writefile tourism_project/data_preparation.py
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from huggingface_hub import hf_hub_download, HfApi, login

def load_data_from_hf():
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
    try:
        file_path = hf_hub_download(
            repo_id="tourism-package-prediction-data",
            filename="tourism.csv",
            repo_type="dataset"
        )
        df = pd.read_csv(file_path)
        print(f"Dataset loaded from Hugging Face, Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise

def clean_data(df):
    df_clean = df.copy()
    if 'Unnamed: 0' in df_clean.columns:
        df_clean = df_clean.drop('Unnamed: 0', axis=1)
    if 'CustomerID' in df_clean.columns:
        df_clean = df_clean.drop('CustomerID', axis=1)
    numerical_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
    print(f"Data cleaned, Final shape: {df_clean.shape}")
    return df_clean

def encode_categorical_features(df):
    df_encoded = df.copy()
    categorical_cols = df_encoded.select_dtypes(include=['object']).columns.tolist()
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le
        print(f"Encoded '{col}': {len(le.classes_)} unique values")
    return df_encoded, label_encoders

def split_and_save_data(df):
    X = df.drop('ProdTaken', axis=1)
    y = df['ProdTaken']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)
    os.makedirs("tourism_project/data", exist_ok=True)
    train_path = "tourism_project/data/train.csv"
    test_path = "tourism_project/data/test.csv"
    train_data.to_csv(train_path, index=False)
    test_data.to_csv(test_path, index=False)
    print(f"Train data shape: {train_data.shape}")
    print(f"Test data shape: {test_data.shape}")
    return train_path, test_path

def upload_to_hf(train_path, test_path):
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
    api = HfApi()
    repo_id = "tourism-package-prediction-data"
    try:
        api.upload_file(path_or_fileobj=train_path, path_in_repo="train.csv", repo_id=repo_id, repo_type="dataset")
        print(f"Train data uploaded to {repo_id}")
        api.upload_file(path_or_fileobj=test_path, path_in_repo="test.csv", repo_id=repo_id, repo_type="dataset")
        print(f"Test data uploaded to {repo_id}")
    except Exception as e:
        print(f"Error uploading to Hugging Face: {str(e)}")
        raise

def main():
    print("DATA PREPARATION PIPELINE")
    df = load_data_from_hf()
    df_clean = clean_data(df)
    df_encoded, label_encoders = encode_categorical_features(df_clean)
    train_path, test_path = split_and_save_data(df_encoded)
    upload_to_hf(train_path, test_path)
    print("DATA PREPARATION COMPLETED SUCCESSFULLY")

if __name__ == "__main__":
    main()

# Model Building

In [ ]:
os.makedirs("tourism_project/model_building", exist_ok=True)
print("Created tourism_project/model_building folder")

In [ ]:
%%writefile tourism_project/model_building/model_training.py
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from huggingface_hub import hf_hub_download, HfApi, login
import joblib
import warnings
warnings.filterwarnings('ignore')

def load_train_test_data():
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
    try:
        train_path = hf_hub_download(repo_id="tourism-package-prediction-data", filename="train.csv", repo_type="dataset")
        train_df = pd.read_csv(train_path)
        test_path = hf_hub_download(repo_id="tourism-package-prediction-data", filename="test.csv", repo_type="dataset")
        test_df = pd.read_csv(test_path)
        X_train = train_df.drop('ProdTaken', axis=1)
        y_train = train_df['ProdTaken']
        X_test = test_df.drop('ProdTaken', axis=1)
        y_test = test_df['ProdTaken']
        print(f"Data loaded - Train: {X_train.shape}, Test: {X_test.shape}")
        return X_train, X_test, y_train, y_test
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1_score': f1_score(y_test, y_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    print(f"{model_name} - Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1_score']:.4f}")
    return metrics

def train_and_log_model(model, model_name, params, X_train, X_test, y_train, y_test):
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        model.fit(X_train, y_train)
        metrics = evaluate_model(model, X_test, y_test, model_name)
        mlflow.log_metrics(metrics)
        mlflow.sklearn.log_model(model, "model")
        return model, metrics

def train_all_models(X_train, X_test, y_train, y_test):
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.set_experiment("Tourism_Package_Prediction")
    results = {}
    
    dt_params = {'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10, 'random_state': 42}
    dt_model, dt_metrics = train_and_log_model(DecisionTreeClassifier(**dt_params), "Decision_Tree", dt_params, X_train, X_test, y_train, y_test)
    results['Decision_Tree'] = {'model': dt_model, 'metrics': dt_metrics}
    
    rf_params = {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 10, 'min_samples_leaf': 5, 'random_state': 42, 'n_jobs': -1}
    rf_model, rf_metrics = train_and_log_model(RandomForestClassifier(**rf_params), "Random_Forest", rf_params, X_train, X_test, y_train, y_test)
    results['Random_Forest'] = {'model': rf_model, 'metrics': rf_metrics}
    
    ada_params = {'n_estimators': 100, 'learning_rate': 0.1, 'random_state': 42}
    ada_model, ada_metrics = train_and_log_model(AdaBoostClassifier(**ada_params), "AdaBoost", ada_params, X_train, X_test, y_train, y_test)
    results['AdaBoost'] = {'model': ada_model, 'metrics': ada_metrics}
    
    gb_params = {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 5, 'random_state': 42}
    gb_model, gb_metrics = train_and_log_model(GradientBoostingClassifier(**gb_params), "Gradient_Boosting", gb_params, X_train, X_test, y_train, y_test)
    results['Gradient_Boosting'] = {'model': gb_model, 'metrics': gb_metrics}
    
    xgb_params = {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 3, 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss'}
    xgb_model, xgb_metrics = train_and_log_model(XGBClassifier(**xgb_params), "XGBoost", xgb_params, X_train, X_test, y_train, y_test)
    results['XGBoost'] = {'model': xgb_model, 'metrics': xgb_metrics}
    
    return results

def select_best_model(results):
    comparison = []
    for model_name, result in results.items():
        metrics = result['metrics']
        comparison.append({'Model': model_name, 'F1-Score': metrics['f1_score']})
    comparison_df = pd.DataFrame(comparison).sort_values('F1-Score', ascending=False)
    best_model_name = comparison_df.iloc[0]['Model']
    best_model = results[best_model_name]['model']
    print(f"Best Model: {best_model_name}")
    return best_model, best_model_name

def save_and_upload_model(model, model_name):
    os.makedirs("tourism_project/model_building", exist_ok=True)
    model_path = f"tourism_project/model_building/{model_name}_model.pkl"
    joblib.dump(model, model_path)
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
    api = HfApi()
    repo_id = "tourism-package-prediction-model"
    api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True, private=False)
    api.upload_file(path_or_fileobj=model_path, path_in_repo=f"{model_name}_model.pkl", repo_id=repo_id, repo_type="model")
    print(f"Model uploaded to {repo_id}")

def main():
    print("MODEL TRAINING PIPELINE")
    X_train, X_test, y_train, y_test = load_train_test_data()
    results = train_all_models(X_train, X_test, y_train, y_test)
    best_model, best_model_name = select_best_model(results)
    save_and_upload_model(best_model, best_model_name)
    print("MODEL TRAINING COMPLETED")

if __name__ == "__main__":
    main()

# Deployment

In [ ]:
os.makedirs("tourism_project/deployment", exist_ok=True)
print("Created tourism_project/deployment folder")

In [ ]:
%%writefile tourism_project/deployment/Dockerfile
FROM python:3.9
WORKDIR /app
COPY . .
RUN pip3 install -r requirements.txt
RUN useradd -m -u 1000 user
USER user
ENV HOME=/home/user PATH=/home/user/.local/bin:$PATH
WORKDIR $HOME/app
COPY --chown=user . $HOME/app
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0", "--server.enableXsrfProtection=false"]

In [ ]:
%%writefile tourism_project/deployment/app.py
import streamlit as st
import pandas as pd
import joblib
from huggingface_hub import hf_hub_download

st.set_page_config(page_title="Tourism Package Predictor", page_icon="✈️", layout="wide")

@st.cache_resource
def load_model():
    try:
        model_path = hf_hub_download(repo_id="tourism-package-prediction-model", filename="XGBoost_model.pkl", repo_type="model")
        return joblib.load(model_path)
    except Exception as e:
        st.error(f"Error loading model: {str(e)}")
        return None

def preprocess_input(data):
    mappings = {
        'TypeofContact': {'Company Invited': 0, 'Self Enquiry': 1},
        'Occupation': {'Salaried': 0, 'Small Business': 1, 'Large Business': 2, 'Free Lancer': 3},
        'Gender': {'Male': 0, 'Female': 1},
        'ProductPitched': {'Basic': 0, 'Standard': 1, 'Deluxe': 2, 'Super Deluxe': 3, 'King': 4},
        'MaritalStatus': {'Single': 0, 'Married': 1, 'Divorced': 2, 'Unmarried': 3},
        'Designation': {'Executive': 0, 'Manager': 1, 'Senior Manager': 2, 'AVP': 3, 'VP': 4}
    }
    processed = data.copy()
    for col, mapping in mappings.items():
        if col in processed.columns:
            processed[col] = processed[col].map(mapping)
    return processed

def main():
    st.title("✈️ Tourism Package Prediction System")
    st.markdown("### Predict whether a customer will purchase the Wellness Tourism Package")
    st.markdown("---")
    
    model = load_model()
    if model is None:
        return
    
    st.sidebar.header("Input Method")
    input_method = st.sidebar.radio("Choose input method:", ["Manual Input", "CSV Upload"])
    
    if input_method == "Manual Input":
        st.header("📝 Enter Customer Details")
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.subheader("Demographics")
            age = st.number_input("Age", 18, 100, 35)
            gender = st.selectbox("Gender", ["Male", "Female"])
            marital_status = st.selectbox("Marital Status", ["Single", "Married", "Divorced", "Unmarried"])
            occupation = st.selectbox("Occupation", ["Salaried", "Small Business", "Large Business", "Free Lancer"])
            designation = st.selectbox("Designation", ["Executive", "Manager", "Senior Manager", "AVP", "VP"])
            monthly_income = st.number_input("Monthly Income", 0, value=20000)
        
        with col2:
            st.subheader("Travel Preferences")
            city_tier = st.selectbox("City Tier", [1, 2, 3])
            preferred_property_star = st.selectbox("Preferred Property Star", [3.0, 4.0, 5.0])
            number_of_person_visiting = st.number_input("Persons Visiting", 1, 10, 2)
            number_of_children_visiting = st.number_input("Children Visiting", 0, 5, 0)
            number_of_trips = st.number_input("Trips per Year", 0.0, 20.0, 2.0)
            passport = st.selectbox("Has Passport?", [0, 1], format_func=lambda x: "Yes" if x == 1 else "No")
            own_car = st.selectbox("Owns Car?", [0, 1], format_func=lambda x: "Yes" if x == 1 else "No")
        
        with col3:
            st.subheader("Interaction Details")
            type_of_contact = st.selectbox("Type of Contact", ["Company Invited", "Self Enquiry"])
            product_pitched = st.selectbox("Product Pitched", ["Basic", "Standard", "Deluxe", "Super Deluxe", "King"])
            duration_of_pitch = st.number_input("Duration of Pitch (min)", 0.0, 60.0, 15.0)
            number_of_followups = st.number_input("Follow-ups", 0.0, 10.0, 3.0)
            pitch_satisfaction_score = st.selectbox("Pitch Satisfaction", [1, 2, 3, 4, 5])
        
        input_data = pd.DataFrame({
            'Age': [age], 'TypeofContact': [type_of_contact], 'CityTier': [city_tier],
            'DurationOfPitch': [duration_of_pitch], 'Occupation': [occupation], 'Gender': [gender],
            'NumberOfPersonVisiting': [number_of_person_visiting], 'NumberOfFollowups': [number_of_followups],
            'ProductPitched': [product_pitched], 'PreferredPropertyStar': [preferred_property_star],
            'MaritalStatus': [marital_status], 'NumberOfTrips': [number_of_trips], 'Passport': [passport],
            'PitchSatisfactionScore': [pitch_satisfaction_score], 'OwnCar': [own_car],
            'NumberOfChildrenVisiting': [number_of_children_visiting], 'Designation': [designation],
            'MonthlyIncome': [monthly_income]
        })
        
        if st.button("🔮 Predict", type="primary"):
            processed_data = preprocess_input(input_data)
            prediction = model.predict(processed_data)[0]
            prediction_proba = model.predict_proba(processed_data)[0]
            st.markdown("---")
            st.header("📊 Prediction Results")
            col1, col2 = st.columns(2)
            with col1:
                if prediction == 1:
                    st.success("✅ Customer is likely to purchase!")
                else:
                    st.error("❌ Customer is unlikely to purchase.")
            with col2:
                st.metric("Confidence", f"{max(prediction_proba)*100:.2f}%")
    
    else:
        st.header("📁 Upload CSV File")
        uploaded_file = st.file_uploader("Choose a CSV file", type="csv")
        if uploaded_file is not None:
            df = pd.read_csv(uploaded_file)
            st.dataframe(df.head())
            if st.button("🔮 Predict All", type="primary"):
                processed_data = preprocess_input(df)
                predictions = model.predict(processed_data)
                df['Prediction'] = predictions
                df['Prediction_Label'] = df['Prediction'].map({0: 'Will Not Purchase', 1: 'Will Purchase'})
                st.header("📊 Results")
                st.metric("Total", len(df))
                st.metric("Likely to Purchase", (predictions == 1).sum())
                st.dataframe(df[['Prediction_Label']])
                st.download_button("📥 Download Results", df.to_csv(index=False), "predictions.csv", "text/csv")

if __name__ == "__main__":
    main()

In [ ]:
%%writefile tourism_project/deployment/requirements.txt
streamlit==1.43.2
pandas==2.2.2
numpy==1.26.4
scikit-learn==1.6.0
xgboost==2.1.4
joblib==1.5.1
huggingface-hub==0.32.6

# Hosting

In [ ]:
%%writefile tourism_project/hosting.py
import os
from huggingface_hub import HfApi, login

def push_to_huggingface_space():
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
        print("Successfully logged in to Hugging Face")
    else:
        print("Error: HF_TOKEN not found")
        return
    
    api = HfApi()
    space_id = "tourism-package-prediction-app"
    
    try:
        api.create_repo(repo_id=space_id, repo_type="space", space_sdk="streamlit", exist_ok=True, private=False)
        print(f"Space '{space_id}' created/verified")
        
        files = [
            ("tourism_project/deployment/Dockerfile", "Dockerfile"),
            ("tourism_project/deployment/app.py", "app.py"),
            ("tourism_project/deployment/requirements.txt", "requirements.txt")
        ]
        
        for local_path, repo_path in files:
            if os.path.exists(local_path):
                api.upload_file(path_or_fileobj=local_path, path_in_repo=repo_path, repo_id=space_id, repo_type="space")
                print(f"Uploaded {repo_path}")
        
        print(f"All files uploaded to {space_id}")
    except Exception as e:
        print(f"Error: {str(e)}")
        raise

if __name__ == "__main__":
    push_to_huggingface_space()

# GitHub Actions Workflow

In [ ]:
os.makedirs("tourism_project/.github/workflows", exist_ok=True)
print("Created tourism_project/.github/workflows folder")

In [ ]:
%%writefile tourism_project/.github/workflows/pipeline.yml
name: MLOps Pipeline

on:
  push:
    branches:
      - main

jobs:
  register-dataset:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Install Dependencies
        run: pip install -r tourism_project/requirements.txt
      - name: Upload Dataset to Hugging Face
        env:
          HF_TOKEN: ${{ secrets.HF_TOKEN }}
        run: python tourism_project/data_registration.py

  data-prep:
    needs: register-dataset
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Install Dependencies
        run: pip install -r tourism_project/requirements.txt
      - name: Run Data Preparation
        env:
          HF_TOKEN: ${{ secrets.HF_TOKEN }}
        run: python tourism_project/data_preparation.py

  model-training:
    needs: data-prep
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Install Dependencies
        run: pip install -r tourism_project/requirements.txt
      - name: Start MLflow Server
        run: |
          nohup mlflow ui --host 0.0.0.0 --port 5000 &
          sleep 5
      - name: Model Training
        env:
          HF_TOKEN: ${{ secrets.HF_TOKEN }}
        run: python tourism_project/model_building/model_training.py

  deploy-hosting:
    runs-on: ubuntu-latest
    needs: [model-training, data-prep, register-dataset]
    steps:
      - uses: actions/checkout@v3
      - name: Install Dependencies
        run: pip install -r tourism_project/requirements.txt
      - name: Push to Hugging Face Space
        env:
          HF_TOKEN: ${{ secrets.HF_TOKEN }}
        run: python tourism_project/hosting.py

# Project Requirements

In [ ]:
%%writefile tourism_project/requirements.txt
huggingface-hub==0.32.6
pandas==2.2.2
numpy==1.26.4
scikit-learn==1.6.0
xgboost==2.1.4
mlflow==3.0.1
joblib==1.5.1

# Summary

This notebook contains all the necessary code to build a complete MLOps pipeline for Tourism Package Prediction:

1. **Data Registration**: Uploads tourism.csv to Hugging Face Dataset Hub
2. **Data Preparation**: Cleans, encodes, and splits data into train/test sets
3. **Model Training**: Trains 5 models (Decision Tree, Random Forest, AdaBoost, Gradient Boosting, XGBoost) with MLflow tracking
4. **Deployment**: Creates Streamlit app with Dockerfile and requirements
5. **Hosting**: Pushes deployment files to Hugging Face Space
6. **CI/CD**: GitHub Actions workflow automates the entire pipeline

## Next Steps:

1. Copy the `tourism.csv` file to `tourism_project/data/` folder
2. Push the `tourism_project` folder to your GitHub repository
3. The GitHub Actions workflow will automatically execute all pipeline steps
4. Access your deployed app on Hugging Face Spaces

**Note**: Remember to replace placeholder Hugging Face user IDs in the scripts with your actual Hugging Face username before deployment.